In [71]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Lectura de los datasets y creación de los dataframes
customers = pd.read_csv('./datasets_clean/customers.csv')
order_items = pd.read_csv('./datasets_clean/order_items.csv')
order_reviews = pd.read_csv('./datasets_clean/order_reviews.csv')
orders_delivered = pd.read_csv('./datasets_clean/orders_delivered.csv')
orders_No_delivered = pd.read_csv('./datasets_clean/orders_No_delivered.csv')
products = pd.read_csv('./datasets_clean/products.csv')
orders = pd.concat([orders_delivered,orders_No_delivered])

In [72]:
#creo algunos datasets sinteticos que me serviran mas adelante
df_inicial = pd.merge(customers,orders,on='customer_id')
df_inicial = pd.merge(df_inicial,order_reviews, on='order_id')
df_inicial = pd.merge(df_inicial,order_items, on='order_id')
df_inicial = pd.merge(df_inicial,products, on='product_id')
#df sera mi listado de productos y puntuaciones, desde el cual creare una matriz para comparar las similitudes
df = df_inicial[['customer_unique_id', 'product_id', 'review_score','order_purchase_timestamp', 'product_category_name']]

#un usuario puede haber comprado el mismo producto varias veces y dejado su reseña asi
#que me aseguro que quede su ultima reseña, tambien puede haber comprado vairas veces el mismo producto en una sola compra
df.drop_duplicates(subset=['customer_unique_id','product_id'],keep='last',inplace=True)
#este dataset lo creo para luego comprobar si hay algun tipo de similitud aunque lo unico que tengo para comparar es
#el tipo de producto y el precio
df_test = df_inicial[['product_id','price']]
df_test = pd.merge(df_test,products,on='product_id')
df_test.drop_duplicates(subset='product_id',keep='last',inplace=True)

C:\Users\Danti\AppData\Local\Temp\ipykernel_7536\4263604408.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['customer_unique_id','product_id'],keep='last',inplace=True)


In [73]:
df_test.product_id.value_counts()[:30]

a9516a079e37a9c9c36b9b78b10169e8    1
6be9d272e87bb43e70975434eaff3ea9    1
26a6bbf34441794b135d4d4642eeece9    1
f17ccf83f9deba389fc0a010257e949a    1
62725a92101c725a6430182a7261416f    1
3fa28edcc7cdfb0ba1d56e70e30ed893    1
108fa3ef6742ead062d9ad59b471ce26    1
6c8ddb5440080718b5110d82c7ae1faf    1
631304377d9ae9aa0a4c1f21b571a502    1
0eae0d3fd2fe4e667213525cd676c1fe    1
b8fbd66f1621afeb9c510b4b9705a5b9    1
00c744ca2f3b0e76ce227b146095d3f9    1
6020b332bb4a8888917d4844eba70bbe    1
fc276546b539ea9ad62827c2a5e2ae42    1
c816e7f68945b875d70f82b82f7b0495    1
0b035ff3ab8827ae8c56022c16712e9c    1
b4546f52515517c7e25a51a13695c1f3    1
8b50377e6ef3606c6e606ca0289d5f34    1
37c871f73c08339b2ee4b6c03337837d    1
faab97003f349e10524a71ed2c2f2bde    1
116a8f521211c179741c051816f2b898    1
f53c1d686fa4b8fe84636440f7a772da    1
51b0fbc0001335d988588a43455a7514    1
70cc46ccdfff951dd7829395bce256ca    1
aae5bd5080ab78211526b58244f1be68    1
8a8e4166f51f98d56dfe007218a11a0e    1
6db02fce12f8

In [74]:
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
le = LabelEncoder()

df['customer_le'] = le.fit_transform(df.customer_unique_id)
df['product_le'] = le.fit_transform(df.product_id)
df_test['product_le'] = le.fit_transform(df_test.product_id)
#reordeno el dataset para ver los ids al principio
df_test = df_test.iloc[:,[10,0,1,2,3,4,5,6,7,8,9]]
df_trasform_producto = df[['product_le','product_id']].drop_duplicates()
products_dicc = dict(zip(df['product_le'].drop_duplicates(),df['product_id'].drop_duplicates()))
N = df['customer_le'].unique().shape[0]
M = df['product_le'].unique().shape[0]
#creo una matriz que contiene todas las opiniones de todos los productos por todos los custemers, a partir
#de esto entrenare a mi modelo para que analise similitudes entre los vectores
X = csr_matrix((df["review_score"].values, (df['product_le'].values, df['customer_le'].values)),shape=(M,N))

C:\Users\Danti\AppData\Local\Temp\ipykernel_7536\1297743115.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['customer_le'] = le.fit_transform(df.customer_unique_id)
C:\Users\Danti\AppData\Local\Temp\ipykernel_7536\1297743115.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['product_le'] = le.fit_transform(df.product_id)


In [75]:

KNN = NearestNeighbors(n_neighbors=10,algorithm='brute',metric='cosine')
KNN.fit(X)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=10)

In [76]:
def find_similar_products(product_id,matrix,k):
    '''
    se le proporciona el id de un producto y te devuelve un listado de productos similares incluyendo el mismo,
    la metrica predeterminada es cosine
    '''
    neighbours_ids= []
    
    product_vector = X[product_id]
    if product_vector.size > 0:
        neighbours = KNN.kneighbors(product_vector, return_distance = True)
        for i in range(1,k+1):
            if neighbours[0].item(i) < 0.35:
                neighbours_ids.append(neighbours[1].item(i))
            
        if len(neighbours_ids) > 0:
            return neighbours_ids
    return "no hay conincidencias"

In [80]:
df_show = df_test.iloc[:5000].loc[df_test.iloc[:5000].product_le.apply(lambda row : find_similar_products(row,X,1) != "no hay conincidencias")]

In [81]:
#creo datasets ya listos para usar asi no tarda mucho en cargar la pagina de streamlit
df_show.to_csv('./datasets_streamlit/df_show.csv',index = False)
df.to_csv('./datasets_streamlit/df.csv',index = False)
df_test.to_csv('./datasets_streamlit/df_test.csv',index = False)